# Theory of BattleSimulator

As of **version 0.3.6** current patch.

This notebook covers the current theoretical underpinning of the BattleSimulator.

Let $\mathbf{u}_i = [x, y]$ define the 2-D cartesian positional coordinates of unit $i$.

## Assignment of Unit Positions

### Drawn from a normal distribution

A gaussian or normal distribution is a common continuous probability that can be used to represent real-valued random variables. Here we draw the 2-D positional vector $\mathbf{u} = [x, y]$ as:

\begin{align}
\mathbf{u} \sim \mathcal{N}(\mu, \Sigma) \tag{1}
\end{align}

where $\mu$ is the mean, and $\Sigma$ the covariance matrix.

### Drawn from a uniform distribution

A uniform distribution is a statistical distribution that bears an equal likelihood of any value in the range $[a, b]$ of being drawn:

\begin{align}
\mathbf{u} \sim \mathcal{U}(a, b) \tag{2}
\end{align}

## Computing the distance from a target unit/location

In order to move the units in 2D space, we have to define how far units are from each other when it comes to calculating whether they are in range to strike. This is determined by the positions and the `range` parameter given to each unit type.

The euclidean distance is defined mathematically as:

\begin{align*}
\mathcal{D}(\textbf{p}, \textbf{q}) = \mathcal{D}(\textbf{q}, \textbf{p}) = \sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2}
\tag{3}
\end{align*}

### Distance from closest enemy

For the primary targeting algorithm, this distance quantity is something to be minimized with respect to all other potential enemies:

\begin{align}
\arg \min \mathcal{D}_i(\mathbf{u}_i, \mathbf{u}_j) \quad \forall j \tag{4}
\end{align}


### Defining directional derivatives

The directional derivative $\nabla_{\textbf{u}} f(x_0, y_0)$ is the rate at which the function $f(x, y)$ changes at a point $(x_0, y_0)$ in the direction $\textbf{u}$. It is defined as:

\begin{align}
\nabla_{\mathbf{u}} f \equiv \nabla f \cdot \frac{\mathbf{u}}{|\mathbf{u}|} \\
= \lim_{h \to 0} \frac{f(\mathbf{x} + h \, \hat{\mathbf{u}}) - f(\mathbf{x})}{h}
\tag{5}
\end{align}

where $\mathbf{u}=[\mathbf{x}, \mathbf{y}]$. In our case, within *Euclidean space*, the directional derivative is respective to the vector $\mathbf{v}$ after normalization, so it's independent of it's magnitude and depends only on the direction.

### The normalized derivative

Since we exclude the magnitude from our calculation; we normalize for it:

\begin{align}
\delta \mathbf{u}_{i}=-\frac{\mathbf{u}_i - \mathbf{u}_j}{\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j)} \tag{6}
\end{align}

This is essentially the *negative normalized direction* of the unit. We make it negative such that the unit moves *towards* the enemy rather than away. Note that it is **essential** that $\mathbf{u}_j - \mathbf{u}_i$ rather than $\mathbf{u}_i - \mathbf{u}_j$, otherwise the unit will move in the *other direction away from the target*.


## Unit movement towards an enemy

Let $s_i$ be the base speed multiplier of unit $i$, and let $z_i(t)$ be the z-coordinate of the height of unit $i$ at time $t$.

The change in unit position given time $t$, measured as $\dot{\mathbf{u}}$ is determined as:

\begin{align}
\dot{\mathbf{u}}_i(t) = \frac{d\mathbf{u}_i}{dt}(\mathbf{u}_i, \mathbf{u}_j) \approx s_i \, \delta \mathbf{u}_{i}\, \left(1-\frac{z_i(t)}{2} \right) \delta t \quad \forall i \\
\tag{7}
\end{align}

where $\delta t$ is the change in time step (default to 1), unit $j$ is the target that unit $i$ wishes to draw towards, and where $\delta \mathbf{u}_{i}$ is our normalized direction for unit $i$. In essense, this is simply the 'speed' of the unit multiplied by the negative direction of the opponent to move in that direction. Note that $ \mathbf{u}_j$ doesn't have to be another Unit, but could be a waypoint for moving a Unit to a particular landmark or location.

Further to this, larger values of $z_i(t)$ will slow down unit movement derivatives, meaning that units on top of a hill move much slower than units in a valley.

**NOTE**: In a future version we may change this to ensure there is only a penalty when *moving uphill*, and an increase for moving *downhill*.

### Movement boundary checks

We ensure that units cannot leave a fixed boundary domain $\Omega \in [x_{min}, x_{max}, y_{min}, y_{max}]$:

\begin{equation}
\mathcal{B}(\mathbf{u})=\begin{cases}
x_{min} & u_x < x_{min} \\
x_{max} & u_x > x_{max} \\
y_{min} & u_y < y_{min} \\
y_{max} & u_y > y_{max} 
\end{cases}
\end{equation}

## Attack rolls

### Hit probability

Let $a_i \in [0, 1]$ be the base accuracy modifier of unit $i$, and let $\mathcal{H} \in \mathbb{R}$ be some global hit penalty. Then a chance to perform a successful hit $h_i(t)$ at time $t$ is:

\begin{align}
h_i(t) = \left[ a_i \left(1-a_j^{-1} \right) \left(1-\frac{\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j)}{\mathcal{H}} \right) \right]
\end{align}

where $a_j^{-1} \in [0, 1]$ is the *dodge chance* of enemy unit $j$. here we see that the further a unit is away, the lower the resulting hit chance will be. The fun doesn't stop there though, the attacker now needs to overcome a random roll from a *uniform distribution*:

### Determining damage output

Let $d_j$ be the base damage modifier for unit $j$ and let $v_i(t)$ be the *vitality* or health of unit $i$ at time $t$. Then the initial damage calculation is:

\begin{align}
\dot{d}_j(t)=d_j \left(1+ \frac{z_j(t)-z_i(t)}{2} \right) \quad \text{s.t.} \,\, h_j(t) > \mathcal{U}(0,1)
\end{align}

once again, height plays a significant role in reducing or increasing initial role damage output. However there is another variable inserted into play: *armour*.

### Armour considerations

Let $A_i(t)$ be the amount of armour unit $i$ has remaining at time $t$, then it's difference for step $t$ is calculated as:

$$
A_i(t+1) = A_i(t) - \dot{d}_j(t)
$$

Then using this difference, we determine whether health is affected:

$$
v_i(t+1)=
\begin{cases}
A_i(t+1) > 0 & v_i(t) \\
A_i(t+1) \le 0 \text{ and } A_i(t) > 0 & v_i(t) - A_i(t+1) \\
\text{else} & v_i(t) - \dot{d}_j(t)
\end{cases}
$$

This calculates such that the overflow over from lost armour is also deducted from the vitality pool.

## Unit in range

Let $r_i$ be the base range modifier of unit $i$.

To determine whether an enemy unit is within range of an attack, an *adjusted* range calculation must be made, factoring in the terrain height of the current unit:

\begin{align}
\bar{r}_i = r_i \left(\frac{z_i^2(t)}{3}+1 \right) \tag{8}
\end{align}

where $z_i(t)$ is the units' current $z$ height coordinate. This means for larger values of $z_i^2(t)$, the adjusted range is exponentially higher.

## Creating a Terrain

Let $\Omega$ be the domain within which the world space is created and units will inhabit. By default we will simulate a pseudo hill-like plain by creating $d$-Gaussian densities on the $z$-plane. This is more formally known as defining a *joint probability density function*.

### Multivariate Gaussian densities

The multivariate normal distribution is said to be "non-degenerate" when the symmetric covariance matrix ${\displaystyle{\boldsymbol {\Sigma }}}$ is positive-definite. In this case the distribution has density:

\begin{align}
\mathcal{G}_{\mathbf{X}}(x_1, \dots, x_d)=\frac{\exp\left(-\frac{1}{2} \left(\mathbf{x}-\mathbf{\mu} \right)^{\text{T}}\mathbf{\Sigma}^{-1} \left(\mathbf{x}-\mathbf{\mu} \right)\right)}{\sqrt{\left(  2 \pi \right)^d |\mathbf{\Sigma}|}}
\end{align}

where $\mathbf{X}$ is a real $d$-dimensional column vector,  $\mathbf{\mu}$ is the mean of $\mathbf{X}$ and $|\mathbf{\Sigma}|$ is the determinant of covariance matrix $\mathbf{\Sigma}$. Thus we define the joint probability density function as the integral sum of all the sub-parts:

\begin{align}
\mathbf{I}=\text{Pr}(X_1, \dots, X_d \in \Omega) = \int_{\Omega}\mathcal{G}_{\mathbf{X}_1, \dots, \mathbf{X}_d}(x_1, \dots, x_d) dx_1, \dots, dx_d \approx \sum_{k=1}^d \mathcal{G}_{\mathbf{X}_k} (x_1, \dots, x_d)
\end{align}

### Discretization of Probability Density

Given discretization of meshgrid $\Omega$ using $\mathbf{I}$, the point at which unit $i$ is closest to node $m$ is:

\begin{align}
z_{im}^{\text{ind}}=\arg \min |\mathbf{u}_i(t)-\mathbf{I} |
\end{align}

such that $z_i(t)=\mathbf{I}\left(z_{im}^{\text{ind}}\right)$. Covariance matrix $\mathbf{\Sigma} \in \mathbb{R}^{2 \times 2}$ in the 2-D case, in addition to $\mathbf{\mu} \in \mathbb{R}^{2}$ whose parameters we sample from a *uniform distribution*. Hereby we refer to a generic dimension as $k$:

\begin{align}
\mu_k \sim \mathcal{U} \left(0, k_{\text{max}}-k_{\text{min}} \right) + k_{\text{min}} \\
\Sigma_{kk}\sim \mathcal{U}(1,2) \psi_k \\
\Sigma_{xy}\equiv \Sigma_{yx} \sim \mathcal{U}(0,1) \mathcal{C}(0, 1)
\end{align}

where 

\begin{align}
\mathcal{C}(0,1)=\begin{cases}
-1 & \mathcal{C} < 0 \\
1 & \mathcal{C} \ge 0
\end{cases}
\end{align}

and acts to flip the direction of the gaussian ellipse. Note that scaling parameters $\psi_k$ are pre-determined by slope $\zeta$ and intercept $\gamma$. These are determined using a linear model:

\begin{align}
\psi_k=\frac{k_{\text{max}} - k_{\text{min}}}{\left(k_{\text{max}} - k_{\text{min}}\right)\zeta + \gamma}
\end{align}

where $\zeta \in [0, 300]$ is determined from the aforementioned range as is $\gamma \in [5, 2]$ using the normal linear model $\psi=k\zeta+\gamma$.

## Deciding how to respond: AI


### Aggressive AI

\begin{align}
\begin{cases}
\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j) > \bar{r}_i & \mathbf{u}_i(t) +\frac{d\mathbf{u}_i}{dt} \\
\text{else} & \text{attack roll}
\end{cases}
\tag{9}
\end{align}

i.e when the distance is greater than the range capability of unit $i$, we need to move closer in order to engage.

### Hit and Run Tactic AI

Currently this AI only activates under the following conditions:


\begin{align}
\begin{cases}
s_i > s_j, \, \bar{r}_i > \bar{r}_j & \text{Hit and Run} \\
\text{else} & \text{Aggressive}
\end{cases}
\end{align}

i.e the speed of unit $i$ is greater than the enemy, and the adjusted range is greater such that this unit cannot simply be ranged down by an opponent. This hit and run is perfect for fast/nimble and longer-range elite units.

\begin{align}
\begin{cases}
\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j) > \bar{r}_i & \mathbf{u}_i(t) + \dot{\mathbf{u}}_i(t) \\
\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j) < \bar{r}_j & \mathbf{u}_i(t) - \dot{\mathbf{u}}_i(t) \\
\text{else} & \text{attack roll}
\end{cases}
\tag{9}
\end{align}

For case one, the unit moves closer to it's opponent when it's out of range, **but** moves away from it's opponent when the oppoenent is within range of itself, leaving the window opportunity when itself is close enough but the opponent is not, maximising potential damage output.